In [1]:
import org.apache.spark.ml.{PipelineModel, Pipeline}
import org.apache.spark.ml.classification.{DecisionTreeClassifier,
  RandomForestClassifier, RandomForestClassificationModel}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, VectorIndexer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import scala.util.Random

Intitializing Scala interpreter ...

Spark Web UI available at http://912db3bbf5c4:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1611569726982)
SparkSession available as 'spark'


import org.apache.spark.ml.{PipelineModel, Pipeline}
import org.apache.spark.ml.classification.{DecisionTreeClassifier, RandomForestClassifier, RandomForestClassificationModel}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, VectorIndexer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import scala.util.Random


In [2]:
val dataWithoutHeader = spark.read.
      option("inferSchema", true).
      option("header", false).
      csv("./data/covtype.data")

dataWithoutHeader: org.apache.spark.sql.DataFrame = [_c0: int, _c1: int ... 53 more fields]


In [3]:
dataWithoutHeader.show()

+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| _c0|_c1|_c2|_c3|_c4| _c5|_c6|_c7|_c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|_c43|_c44|_c45|_c46|_c47|_c48|_c49|_c50|_c51|_c52|_c53|_c54|
+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|2596| 51|  3|258|  0| 510|221|232|148|6279|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0| 

In [4]:
dataWithoutHeader.head()

res1: org.apache.spark.sql.Row = [2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5]


In [5]:
val colNames = Seq(
        "Elevation", "Aspect", "Slope",
        "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
        "Horizontal_Distance_To_Roadways",
        "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
        "Horizontal_Distance_To_Fire_Points"
      ) ++ (
        (0 until 4).map(i => s"Wilderness_Area_$i")
      ) ++ (
        (0 until 40).map(i => s"Soil_Type_$i")
      ) ++ Seq("Cover_Type")

colNames: Seq[String] = List(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, Soil_...


In [6]:
(0 until 4).map(i => s"Wilderness_Area_$i")

res2: scala.collection.immutable.IndexedSeq[String] = Vector(Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3)


## 1. MLlib의 모든 api는 int가 아닌 Double만 받는다.

In [7]:
val data = dataWithoutHeader.toDF(colNames:_*).
  withColumn("Cover_Type", $"Cover_Type".cast("double"))

data: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 53 more fields]


In [8]:
val Array(trainData, testData) = data.randomSplit(Array(0.9, 0.1))
trainData.cache()
testData.cache()

trainData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Elevation: int, Aspect: int ... 53 more fields]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Elevation: int, Aspect: int ... 53 more fields]
res3: testData.type = [Elevation: int, Aspect: int ... 53 more fields]


## 2. assembler : CSR 읽는 방법 체크

- MLlib 파이프라인 API중 Transformer의 예시
- 데이터프레임을 다른 데이터프레임으로 변환
- 이런 변환들을 묶어 하나의 파이프라인으로 구성

In [9]:
val inputCols = trainData.columns.filter(_ != "Cover_Type")
val assembler = new VectorAssembler().
  setInputCols(inputCols).
  setOutputCol("featureVector")

inputCols: Array[String] = Array(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, S...


In [10]:
trainData.head(3)

res4: Array[org.apache.spark.sql.Row] = Array([1874,18,14,0,0,90,208,209,135,793,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0], [1879,28,19,30,12,95,209,196,117,778,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0], [1888,33,22,150,46,108,209,185,103,735,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0])


In [11]:
val assembledTrainData = assembler.transform(trainData)
assembledTrainData.select("featureVector").show(truncate = false)

+-----------------------------------------------------------------------------------------------------+
|featureVector                                                                                        |
+-----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,5,6,7,8,9,13,18],[1874.0,18.0,14.0,90.0,208.0,209.0,135.0,793.0,1.0,1.0])                 |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1879.0,28.0,19.0,30.0,12.0,95.0,209.0,196.0,117.0,778.0,1.0,1.0])   |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1888.0,33.0,22.0,150.0,46.0,108.0,209.0,185.0,103.0,735.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1889.0,28.0,22.0,150.0,23.0,120.0,205.0,185.0,108.0,759.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1889.0,353.0,30.0,95.0,39.0,67.0,153.0,172.0,146.0,600.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1896.0,337.0,12.0,30.0,6.0,175.0,195.0,224.0,168.0,732.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1899.0,355.0,22.0,153.0,43.0,1

assembledTrainData: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 54 more fields]


In [12]:
assembledTrainData

res6: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 54 more fields]


In [13]:
assembledTrainData.columns.foreach(println)

Elevation
Aspect
Slope
Horizontal_Distance_To_Hydrology
Vertical_Distance_To_Hydrology
Horizontal_Distance_To_Roadways
Hillshade_9am
Hillshade_Noon
Hillshade_3pm
Horizontal_Distance_To_Fire_Points
Wilderness_Area_0
Wilderness_Area_1
Wilderness_Area_2
Wilderness_Area_3
Soil_Type_0
Soil_Type_1
Soil_Type_2
Soil_Type_3
Soil_Type_4
Soil_Type_5
Soil_Type_6
Soil_Type_7
Soil_Type_8
Soil_Type_9
Soil_Type_10
Soil_Type_11
Soil_Type_12
Soil_Type_13
Soil_Type_14
Soil_Type_15
Soil_Type_16
Soil_Type_17
Soil_Type_18
Soil_Type_19
Soil_Type_20
Soil_Type_21
Soil_Type_22
Soil_Type_23
Soil_Type_24
Soil_Type_25
Soil_Type_26
Soil_Type_27
Soil_Type_28
Soil_Type_29
Soil_Type_30
Soil_Type_31
Soil_Type_32
Soil_Type_33
Soil_Type_34
Soil_Type_35
Soil_Type_36
Soil_Type_37
Soil_Type_38
Soil_Type_39
Cover_Type
featureVector


## Model 

In [14]:
val classifier = new DecisionTreeClassifier().
  setSeed(Random.nextLong()).
  setLabelCol("Cover_Type").
  setFeaturesCol("featureVector").
  setPredictionCol("prediction")

val model = classifier.fit(assembledTrainData)
println(model.toDebugString)

DecisionTreeClassificationModel: uid=dtc_3e08ba901394, depth=5, numNodes=47, numClasses=8, numFeatures=54
  If (feature 0 <= 3048.5)
   If (feature 0 <= 2541.5)
    If (feature 10 <= 0.5)
     If (feature 0 <= 2408.5)
      If (feature 3 <= 15.0)
       Predict: 4.0
      Else (feature 3 > 15.0)
       Predict: 3.0
     Else (feature 0 > 2408.5)
      Predict: 3.0
    Else (feature 10 > 0.5)
     If (feature 9 <= 5472.5)
      Predict: 2.0
     Else (feature 9 > 5472.5)
      If (feature 5 <= 542.0)
       Predict: 2.0
      Else (feature 5 > 542.0)
       Predict: 5.0
   Else (feature 0 > 2541.5)
    If (feature 0 <= 2956.5)
     If (feature 15 <= 0.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
     Else (feature 15 > 0.5)
      Predict: 3.0
    Else (feature 0 > 2956.5)
     If (feature 3 <= 191.0)
      If (feature 36 <= 0.5)
       Predict: 2.0
      Else (feature 36 > 0.5)
       Predict: 1.0
     Else (feature 3 > 191.0)
   

classifier: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_3e08ba901394
model: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel: uid=dtc_3e08ba901394, depth=5, numNodes=47, numClasses=8, numFeatures=54


## 인덱스 0은 의미 없음

In [15]:
assembledTrainData.select("Cover_Type").distinct().collect()

res9: Array[org.apache.spark.sql.Row] = Array([7.0], [1.0], [4.0], [3.0], [2.0], [6.0], [5.0])


In [16]:
val predictions = model.transform(assembledTrainData)

predictions.select("Cover_Type", "prediction", "probability").
show(truncate = false)

+----------+----------+--------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                       |
+----------+----------+--------------------------------------------------------------------------------------------------+
|6.0       |4.0       |[0.0,0.0,0.031062124248496994,0.29458917835671344,0.48396793587174347,0.0,0.1903807615230461,0.0] |
|6.0       |3.0       |[0.0,0.0,0.024144129422338425,0.6469482800882425,0.059686248876542204,0.0,0.26922134161287686,0.0]|
|6.0       |3.0       |[0.0,0.0,0.024144129422338425,0.6469482800882425,0.059686248876542204,0.0,0.26922134161287686,0.0]|
|6.0       |3.0       |[0.0,0.0,0.024144129422338425,0.6469482800882425,0.059686248876542204,0.0,0.26922134161287686,0.0]|
|6.0       |3.0       |[0.0,0.0,0.024144129422338425,0.6469482800882425,0.059686248876542204,0.0,0.26922134161287686,0.0]|
|6.0       |3.0 

predictions: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 57 more fields]


In [17]:
model.featureImportances.toArray.zip(inputCols).
    sorted.reverse.foreach(println)

(0.8115508430225156,Elevation)
(0.04176007601079249,Horizontal_Distance_To_Hydrology)
(0.02699788134250767,Soil_Type_1)
(0.02635013526940948,Soil_Type_3)
(0.025116576702914076,Hillshade_Noon)
(0.024676057165493523,Soil_Type_31)
(0.02037603230938446,Wilderness_Area_0)
(0.011826889934156499,Wilderness_Area_2)
(0.005591085323383825,Soil_Type_22)
(0.0025495624985275604,Horizontal_Distance_To_Roadways)
(0.0025348532961281707,Hillshade_9am)
(6.700071247868423E-4,Horizontal_Distance_To_Fire_Points)
(0.0,Wilderness_Area_3)
(0.0,Wilderness_Area_1)
(0.0,Vertical_Distance_To_Hydrology)
(0.0,Soil_Type_9)
(0.0,Soil_Type_8)
(0.0,Soil_Type_7)
(0.0,Soil_Type_6)
(0.0,Soil_Type_5)
(0.0,Soil_Type_4)
(0.0,Soil_Type_39)
(0.0,Soil_Type_38)
(0.0,Soil_Type_37)
(0.0,Soil_Type_36)
(0.0,Soil_Type_35)
(0.0,Soil_Type_34)
(0.0,Soil_Type_33)
(0.0,Soil_Type_32)
(0.0,Soil_Type_30)
(0.0,Soil_Type_29)
(0.0,Soil_Type_28)
(0.0,Soil_Type_27)
(0.0,Soil_Type_26)
(0.0,Soil_Type_25)
(0.0,Soil_Type_24)
(0.0,Soil_Type_23)
(0.0,S

In [18]:
val evaluator = new MulticlassClassificationEvaluator().
  setLabelCol("Cover_Type").
  setPredictionCol("prediction")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = MulticlassClassificationEvaluator: uid=mcEval_5464f6fcaca5, metricName=f1, metricLabel=0.0, beta=1.0, eps=1.0E-15


In [19]:
val accuracy = evaluator.setMetricName("accuracy").evaluate(predictions)
val f1 = evaluator.setMetricName("f1").evaluate(predictions)
println(accuracy)
println(f1)

0.703284812473952
0.6864919747345175


accuracy: Double = 0.703284812473952
f1: Double = 0.6864919747345175


## Confusion Matrix. 여전히 RDD만 지원하는듯? [Docs](https://spark.apache.org/docs/latest/api/java/org/apache/spark/mllib/evaluation/MulticlassMetrics.html)

In [20]:
val predictionRDD = predictions.
    select("prediction", "Cover_Type").
    as[(Double,Double)].rdd
val multiclassMetrics = new MulticlassMetrics(predictionRDD)
println(multiclassMetrics.confusionMatrix)

131309.0  54190.0   197.0    0.0    0.0   0.0  5046.0   
50731.0   196235.0  7353.0   62.0   16.0  0.0  701.0    
0.0       2557.0    29084.0  588.0  0.0   0.0  0.0      
0.0       0.0       1505.0   966.0  0.0   0.0  0.0      
4.0       7693.0    768.0    0.0    77.0  0.0  0.0      
0.0       3365.0    11813.0  380.0  0.0   0.0  0.0      
8161.0    15.0      59.0     0.0    0.0   0.0  10199.0  


predictionRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[118] at rdd at <console>:40
multiclassMetrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@193dd009


In [21]:
// 여기까지는 Dataset
val predictionDataset = predictions.
      select("prediction", "Cover_Type").
      as[(Double,Double)]

predictionDataset: org.apache.spark.sql.Dataset[(Double, Double)] = [prediction: double, Cover_Type: double]


In [22]:
val confusionMatrix = predictions.
  groupBy("Cover_Type").
  pivot("prediction", (1 to 7)).
  count().
  na.fill(0.0).
  orderBy("Cover_Type")

confusionMatrix.show()

+----------+------+------+-----+---+---+---+-----+
|Cover_Type|     1|     2|    3|  4|  5|  6|    7|
+----------+------+------+-----+---+---+---+-----+
|       1.0|131309| 54190|  197|  0|  0|  0| 5046|
|       2.0| 50731|196235| 7353| 62| 16|  0|  701|
|       3.0|     0|  2557|29084|588|  0|  0|    0|
|       4.0|     0|     0| 1505|966|  0|  0|    0|
|       5.0|     4|  7693|  768|  0| 77|  0|    0|
|       6.0|     0|  3365|11813|380|  0|  0|    0|
|       7.0|  8161|    15|   59|  0|  0|  0|10199|
+----------+------+------+-----+---+---+---+-----+



confusionMatrix: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Cover_Type: double, 1: bigint ... 6 more fields]


## Baseline

In [23]:
def classProbabilities(data: DataFrame): Array[Double] = {
    val total = data.count()
    data.groupBy("Cover_Type").count().
    orderBy("Cover_Type").
    select("count").as[Double].
    map(_ / total).
    collect()
}

classProbabilities: (data: org.apache.spark.sql.DataFrame)Array[Double]


In [24]:
val trainPriorProbabilities = classProbabilities(trainData)
val testPriorProbabilities = classProbabilities(testData)

trainPriorProbabilities: Array[Double] = Array(0.36465586131216615, 0.4876900782680844, 0.061614609022815126, 0.004723996987041987, 0.01633038537568298, 0.02974340150724372, 0.035241667526965594)
testPriorProbabilities: Array[Double] = Array(0.3641478822189237, 0.48677897062377024, 0.060840898891918946, 0.004763712934516207, 0.01641409782871345, 0.03122303151644862, 0.03583140598570886)


In [25]:
val accuracy = trainPriorProbabilities.zip(testPriorProbabilities).map {
  case (trainProb, cvProb) => trainProb * cvProb
}.sum

accuracy: Double = 0.3764166120842551


In [49]:
// case 없으면 에러남. https://jdm.kr/blog/85#멀티_바인딩(destructuring_bind)
val accuracy = trainPriorProbabilities.zip(testPriorProbabilities).map {
 (trainProb, cvProb) => trainProb * cvProb
}.sum

<console>: 41: error: missing parameter type

## 튜닝

In [26]:
val inputCols = trainData.columns.filter(_ != "Cover_Type")
val assembler = new VectorAssembler().
  setInputCols(inputCols).
  setOutputCol("featureVector")

val classifier = new DecisionTreeClassifier().
  setSeed(Random.nextLong()).
  setLabelCol("Cover_Type").
  setFeaturesCol("featureVector").
  setPredictionCol("prediction")

val pipeline = new Pipeline().setStages(Array(assembler, classifier))

inputCols: Array[String] = Array(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, S...


In [27]:
val paramGrid = new ParamGridBuilder().
  addGrid(classifier.maxDepth, Seq(1, 20)).
  build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	dtc_7201084bac8d-maxDepth: 1
}, {
	dtc_7201084bac8d-maxDepth: 20
})


In [28]:
val multiclassEval = new MulticlassClassificationEvaluator().
  setLabelCol("Cover_Type").
  setPredictionCol("prediction").
  setMetricName("accuracy")

multiclassEval: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = MulticlassClassificationEvaluator: uid=mcEval_68449c92b7b6, metricName=accuracy, metricLabel=0.0, beta=1.0, eps=1.0E-15


In [29]:
val validator = new TrainValidationSplit().
  setSeed(Random.nextLong()).
  setEstimator(pipeline).  // 어떤 데이터, 모델을
  setEvaluator(multiclassEval).  // 어떤 지표로
  setEstimatorParamMaps(paramGrid).  // 어떤 하이퍼파라미터를
  setTrainRatio(0.9)

val validatorModel = validator.fit(trainData)

validator: org.apache.spark.ml.tuning.TrainValidationSplit = tvs_f86e6a833f32
validatorModel: org.apache.spark.ml.tuning.TrainValidationSplitModel = TrainValidationSplitModel: uid=tvs_f86e6a833f32, bestModel=pipeline_bcc309c6a2c0, trainRatio=0.9


### 제일 좋은 모델 찾기

In [30]:
val paramsAndMetrics = validatorModel.validationMetrics.
  zip(validatorModel.getEstimatorParamMaps).sortBy(-_._1)

paramsAndMetrics.foreach { case (metric, params) =>
    println(metric)
    println(params)
    println()
}

0.9033259848557462
{
	dtc_7201084bac8d-maxDepth: 20
}

0.6296942394325697
{
	dtc_7201084bac8d-maxDepth: 1
}



paramsAndMetrics: Array[(Double, org.apache.spark.ml.param.ParamMap)] =
Array((0.9033259848557462,{
	dtc_7201084bac8d-maxDepth: 20
}), (0.6296942394325697,{
	dtc_7201084bac8d-maxDepth: 1
}))


In [31]:
val bestModel = validatorModel.bestModel
println(bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap)

{
	dtc_7201084bac8d-cacheNodeIds: false,
	dtc_7201084bac8d-checkpointInterval: 10,
	dtc_7201084bac8d-featuresCol: featureVector,
	dtc_7201084bac8d-impurity: gini,
	dtc_7201084bac8d-labelCol: Cover_Type,
	dtc_7201084bac8d-leafCol: ,
	dtc_7201084bac8d-maxBins: 32,
	dtc_7201084bac8d-maxDepth: 20,
	dtc_7201084bac8d-maxMemoryInMB: 256,
	dtc_7201084bac8d-minInfoGain: 0.0,
	dtc_7201084bac8d-minInstancesPerNode: 1,
	dtc_7201084bac8d-minWeightFractionPerNode: 0.0,
	dtc_7201084bac8d-predictionCol: prediction,
	dtc_7201084bac8d-probabilityCol: probability,
	dtc_7201084bac8d-rawPredictionCol: rawPrediction,
	dtc_7201084bac8d-seed: -667345610833198024
}


bestModel: org.apache.spark.ml.Model[_] = pipeline_bcc309c6a2c0


In [32]:
println(validatorModel.validationMetrics.max)

0.9033259848557462


In [33]:
val testAccuracy = multiclassEval.evaluate(bestModel.transform(testData))
println(testAccuracy)

val trainAccuracy = multiclassEval.evaluate(bestModel.transform(trainData))
println(trainAccuracy)

0.8998757292277952
0.9354546393053373


testAccuracy: Double = 0.8998757292277952
trainAccuracy: Double = 0.9354546393053373


## udf 로 범주형 피쳐 다르게 해보기

In [ ]:
udf

In [75]:
val unhotUDF = udf((vec: Vector) => vec.toArray.indexOf(1.0).toDouble)

unhotUDF: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5269/0x000000084180c040@2758d837,DoubleType,List(Some(class[value[0]: vector])),None,false,true)


In [76]:
val wildernessCols = (0 until 4).map(i => s"Wilderness_Area_$i").toArray

val wildernessAssembler = new VectorAssembler().
  setInputCols(wildernessCols).
  setOutputCol("wilderness")

wildernessCols: Array[String] = Array(Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3)
wildernessAssembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_3bc447f94e7c, handleInvalid=error, numInputCols=4


In [77]:
val wilderness = wildernessAssembler.transform(assembledTrainData).withColumn("abc", unhotUDF($"wilderness"))

wilderness: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 56 more fields]


In [87]:
wilderness.select("abc").head(50)

res47: Array[org.apache.spark.sql.Row] = Array([3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0], [3.0])


In [89]:
wilderness.groupBy("abc").count().show()

+---+------+
|abc| count|
+---+------+
|0.0|234948|
|1.0| 26853|
|3.0| 33204|
|2.0|228069|
+---+------+



In [90]:
val shuffledDF = wilderness.orderBy(rand())

shuffledDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Elevation: int, Aspect: int ... 56 more fields]


In [92]:
shuffledDF.select("abc").head(50)

res51: Array[org.apache.spark.sql.Row] = Array([0.0], [2.0], [1.0], [2.0], [2.0], [2.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [3.0], [2.0], [0.0], [2.0], [3.0], [2.0], [2.0], [0.0], [2.0], [0.0], [0.0], [2.0], [0.0], [0.0], [0.0], [0.0], [2.0], [0.0], [3.0], [2.0], [0.0], [2.0], [1.0], [2.0], [0.0], [2.0], [2.0], [2.0], [1.0], [0.0], [2.0], [2.0], [0.0], [1.0], [0.0], [0.0], [2.0])


In [83]:
wildernessAssembler.transform(assembledTrainData).select("Wilderness_Area_0").head(20)

res43: Array[org.apache.spark.sql.Row] = Array([0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0])


In [84]:
wildernessAssembler.transform(assembledTrainData).select("Wilderness_Area_1").head(20)

res44: Array[org.apache.spark.sql.Row] = Array([0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0])


In [85]:
wildernessAssembler.transform(assembledTrainData).select("Wilderness_Area_2").head(20)

res45: Array[org.apache.spark.sql.Row] = Array([0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0])


In [86]:
wildernessAssembler.transform(assembledTrainData).select("Wilderness_Area_3").head(20)

res46: Array[org.apache.spark.sql.Row] = Array([1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1])


In [57]:
def unencodeOneHot(data: DataFrame): DataFrame = {
    val wildernessCols = (0 until 4).map(i => s"Wilderness_Area_$i").toArray

    val wildernessAssembler = new VectorAssembler().
      setInputCols(wildernessCols).
      setOutputCol("wilderness")

    val unhotUDF = udf((vec: Vector) => vec.toArray.indexOf(1.0).toDouble)

    val withWilderness = wildernessAssembler.transform(data).
      drop(wildernessCols:_*).
      withColumn("wilderness", unhotUDF($"wilderness"))

    val soilCols = (0 until 40).map(i => s"Soil_Type_$i").toArray

    val soilAssembler = new VectorAssembler().
      setInputCols(soilCols).
      setOutputCol("soil")

    soilAssembler.transform(withWilderness).
      drop(soilCols:_*).
      withColumn("soil", unhotUDF($"soil"))
}

unencodeOneHot: (data: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [62]:
val unencTrainData = unencodeOneHot(trainData)

unencTrainData: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 11 more fields]


In [63]:
val inputCols = unencTrainData.columns.filter(_ != "Cover_Type")
val assembler = new VectorAssembler().
  setInputCols(inputCols).
  setOutputCol("featureVector")

val indexer = new VectorIndexer().
  setMaxCategories(40).
  setInputCol("featureVector").
  setOutputCol("indexedVector")

val classifier = new RandomForestClassifier().
  setSeed(Random.nextLong()).
  setLabelCol("Cover_Type").
  setFeaturesCol("indexedVector").
  setPredictionCol("prediction").
  setImpurity("entropy").
  setMaxDepth(20).
  setMaxBins(300)

val pipeline = new Pipeline().setStages(Array(assembler, indexer, classifier))

inputCols: Array[String] = Array(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, wilderness, soil)
assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_68f197e89966, handleInvalid=error, numInputCols=12
indexer: org.apache.spark.ml.feature.VectorIndexer = vecIdx_67d0ee63ca7b
classifier: org.apache.spark.ml.classification.RandomForestClassifier = rfc_1e23e171732d
pipeline: org.apache.spark.ml.Pipeline = pipeline_8dc1bf5baba2


## RandomForest

In [64]:
val classifier = new RandomForestClassifier().
      setSeed(Random.nextLong()).
      setLabelCol("Cover_Type").
      setFeaturesCol("indexedVector").
      setPredictionCol("prediction").
      setImpurity("entropy").
      setMaxDepth(20).
      setMaxBins(300)

classifier: org.apache.spark.ml.classification.RandomForestClassifier = rfc_f0ef07a4ed84


## Microsoft mmlspark LGBM
https://github.com/Azure/mmlspark/blob/master/docs/lightgbm.md